In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li pre{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터 수집 - 동적</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- Selenium docs : https://selenium-python.readthedocs.io/
`pip install selenium`
- import에서 에러날 경우
`pip install requests`나 `conda install urllib3==1.26.18`

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [4]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")
elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')